In [34]:
import numpy as np, matplotlib.pyplot as plt

def source(x, t, fn):
    f = np.empty((x.shape[0], t.shape[0]))
    for i in range(x.shape[0]):
        f[i, :] = fn(x[i], t)
    return f

class Wave_Equation:
    def __init__(self):
        pass

    def make_grid(self, x0=0, xn=1, nx=100,\
                        t0=0, tn=1, nt=100):
        self.x = np.linspace(x0, xn, nx)
        self.t = np.linspace(t0, tn, nt)
        self.hx = (xn - x0)/nx
        self.ht = (tn - t0)/nt
        self.s = self.ht**2/self.hx**2
        self.nt, self.nx = nt, nx

    def solve(self, gamma, eta, phi, psi, f):
        gamma = gamma(self.t)
        eta = eta(self.t)
        phi = phi(self.x)
        psi = psi(self.x)
        f = source(self.x, self.t, f)
        nx, nt = self.nx, self.nt
        u = np.zeros((nx, nt)) 
        u[:, 0] = phi
        u[0, 0] = gamma[0] 
        u[-1,0] = eta[0]
        s, ht = self.s, self.ht
        
        # Setting second time step
        for i in range(1, nx-1):
            u[i, 1] = 0.5*s*(phi[i+1] + phi[i-1]) + (1-s)*phi[i] + ht*psi[i] + 0.5*f[i, 1]
        u[0, 1] = gamma[1]
        u[-1, 1] = eta[1]
        
        #solve
        for i in range(1, nt-1):
            u[0, i] = gamma[i]
            u[-1, i] = eta[i]
            for j in range(1, nx-1):
                u[j, i+1] = s*(u[j+1, i] + u[j-1,i]) + 2*(1-s)*u[j,i] - u[j,i-1] + f[j, i]

        return u


In [92]:

a, b, x0 = 7, 4, 0
rho = lambda x: x*0
WE = Wave_Equation()
WE.make_grid(x0 = -5, xn = 5, nx=200, t0=0, tn=10, nt=300)
x, t = WE.x, WE.t

sol_analytic = np.empty((WE.nx, WE.nt))

for i in range(WE.nt):
    sol_analytic[:, i] = 0.5*(rho(x+t[i]) + rho(x-t[i])) 

def f(t):
    return(t*0)
def g(t):
    return(t*0)
def psi(x):
    return(np.exp(-x))
sol_numerical = WE.solve(f, g, rho, psi)

WE.s


0.44444444444444436

In [104]:
%matplotlib notebook
from animator import Animator
movie = Animator(x, t, sol_numerical, sol_analytic, skip_frames=4)
movie.animate() 
from IPython.display import HTML
HTML(movie.ani.to_jshtml()) 

<IPython.core.display.Javascript object>